### Import & Configuration

In [1]:
import os
# --- FIX FOR OMP: Error #15 ---
# Variabel lingkungan ini adalah solusi untuk konflik antara runtime OpenMP.
# Ini memberitahu sistem untuk mengizinkan beberapa pustaka OpenMP untuk dimuat.
# Letakkan ini SEBELUM mengimpor torch, pandas, atau numpy.
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import easyocr
import pandas as pd
from tqdm.notebook import tqdm # Gunakan tqdm.notebook untuk tampilan yang lebih baik di Jupyter
import re
import torch
import matplotlib.pyplot as plt
import seaborn as sns

# --- CONFIGURATION ---
# The root directory containing all your experiment folders (exp1, exp2, etc.)
ROOT_DIR = "dataset_eval"
OUTPUT_CSV = "ocr_accuracy_results.csv"
OUTPUT_DIR_PLOTS = "output_plots" # Folder untuk menyimpan visualisasi

print("✅ Pustaka dan konfigurasi berhasil dimuat.")

✅ Pustaka dan konfigurasi berhasil dimuat.


### Def Library Cell

In [2]:
def get_ground_truth(filename):
    """Mengekstrak teks onomatope yang diharapkan dari nama file."""
    # Normalisasi nama file untuk pencocokan yang konsisten
    name_part = filename.split('_')[0].lower()
    
    if "byur" in name_part:
        return "byur"
    if "dutt" in name_part: # Mencocokkan Dutt, Duttt, dll.
        return "duuttt"
    if "gluduk" in name_part:
        return "gluduk gluduk gluduk"
    if "gong" in name_part: # Mencocokkan Gong, Gongg, dll.
        return "gonggg"
    if "ting" in name_part:
        return "ting"
    if "toktok" in name_part:
        return "tok tok tok"
    return None

def normalize_text(text):
    """Membersihkan dan menormalkan teks untuk perbandingan."""
    # Ubah ke huruf kecil, hapus semua karakter non-alfanumerik kecuali spasi
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    # Gabungkan beberapa spasi menjadi satu
    text = re.sub(r'\s+', ' ', text).strip()
    return text

print("✅ Fungsi bantuan (get_ground_truth, normalize_text) telah didefinisikan.")

✅ Fungsi bantuan (get_ground_truth, normalize_text) telah didefinisikan.


### Model Initiation and preparation

In [3]:
print("Initializing EasyOCR... This might take a moment.")
# Inisialisasi pembaca OCR untuk bahasa Inggris.
# Gunakan gpu=False jika Anda mengalami crash karena memori GPU tidak cukup.
reader = easyocr.Reader(['en']) 

# Siapkan list kosong untuk menampung semua hasil dari semua folder
all_results = []

print("✅ EasyOCR ready to be used.")

Initializing EasyOCR... This might take a moment.
✅ EasyOCR ready to be used.


#### Folder by Folder Processing

In [4]:
# Cari semua folder eksperimen di dalam direktori root
experiment_folders = [d for d in os.listdir(ROOT_DIR) if os.path.isdir(os.path.join(ROOT_DIR, d))]

print(f"Ditemukan {len(experiment_folders)} folder eksperimen untuk diproses: {experiment_folders}")

# Iterasi dan proses setiap folder
for experiment_name in experiment_folders:
    dirpath = os.path.join(ROOT_DIR, experiment_name)
    image_files = [f for f in os.listdir(dirpath) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if not image_files:
        continue

    print(f"\n▶️ Memulai pemrosesan untuk eksperimen: {experiment_name}...")
    
    for filename in tqdm(image_files, desc=f"Membaca teks dari {experiment_name}"):
        image_path = os.path.join(dirpath, filename)
        ground_truth = get_ground_truth(filename)
        
        if ground_truth is None:
            continue

        try:
            ocr_result = reader.readtext(image_path, detail=0)
            recognized_text = " ".join(ocr_result)
            normalized_recognized = normalize_text(recognized_text)
            normalized_truth = normalize_text(ground_truth)
            is_correct = 1 if normalized_recognized == normalized_truth else 0
            
            all_results.append({
                "experiment": experiment_name, "filename": filename,
                "ground_truth": ground_truth, "recognized_text": recognized_text,
                "is_correct": is_correct
            })
        except Exception as e:
            all_results.append({
                "experiment": experiment_name, "filename": filename,
                "ground_truth": ground_truth, "recognized_text": f"ERROR: {e}",
                "is_correct": 0
            })
        finally:
            # FIX PENTING: Bersihkan cache memori GPU setelah setiap gambar
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    print(f"✅ Selesai memproses {experiment_name}.")

print("\n🎉 Semua folder telah berhasil diproses!")

Ditemukan 5 folder eksperimen untuk diproses: ['exp1_sd15_2050', 'exp2_sd15_3050', 'exp3_sd15_3050_v2', 'exp4_sd15_3050_v3', 'exp5_sdxl_a6000']

▶️ Memulai pemrosesan untuk eksperimen: exp1_sd15_2050...


Membaca teks dari exp1_sd15_2050:   0%|          | 0/60 [00:00<?, ?it/s]

✅ Selesai memproses exp1_sd15_2050.

▶️ Memulai pemrosesan untuk eksperimen: exp2_sd15_3050...


Membaca teks dari exp2_sd15_3050:   0%|          | 0/60 [00:00<?, ?it/s]

✅ Selesai memproses exp2_sd15_3050.

▶️ Memulai pemrosesan untuk eksperimen: exp3_sd15_3050_v2...


...

✅ Selesai memproses exp3_sd15_3050_v2.

▶️ Memulai pemrosesan untuk eksperimen: exp4_sd15_3050_v3...


Membaca teks dari exp4_sd15_3050_v3:   0%|          | 0/60 [00:00<?, ?it/s]

✅ Selesai memproses exp4_sd15_3050_v3.

▶️ Memulai pemrosesan untuk eksperimen: exp5_sdxl_a6000...


Membaca teks dari exp5_sdxl_a6000:   0%|          | 0/60 [00:00<?, ?it/s]

✅ Selesai memproses exp5_sdxl_a6000.

🎉 Semua folder telah berhasil diproses!


### Final Analysis and Storing

In [6]:
if not all_results:
    print("Tidak ada gambar yang diproses. Harap jalankan Sel 4 terlebih dahulu.")
else:
    # Buat dan simpan DataFrame
    df = pd.DataFrame(all_results)
    df.to_csv(OUTPUT_CSV, index=False)
    
    # --- SECTION VISUALIZATION ---
    print("\n--- Generating Visualizations ---")
    os.makedirs(OUTPUT_DIR_PLOTS, exist_ok=True)

    # 1. Visualisasi Akurasi per Eksperimen (Bar Chart)
    accuracy_by_exp = df.groupby('experiment')['is_correct'].mean().reset_index()
    accuracy_by_exp['is_correct'] *= 100
    accuracy_by_exp.rename(columns={'is_correct': 'accuracy_%'}, inplace=True)
    accuracy_by_exp.sort_values('accuracy_%', ascending=False, inplace=True)

    plt.figure(figsize=(12, 7))
    sns.barplot(data=accuracy_by_exp, x='experiment', y='accuracy_%', palette='viridis')
    plt.title('Performa OCR Terbaik per Eksperimen', fontsize=16)
    plt.xlabel('Eksperimen', fontsize=12)
    plt.ylabel('Akurasi (%)', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plot_path_bar = os.path.join(OUTPUT_DIR_PLOTS, 'best_experiment_ocr_performance.png')
    plt.savefig(plot_path_bar)
    plt.close()
    print(f"Bar chart performa terbaik disimpan di: {plot_path_bar}")
    
    # 2. Visualisasi Keberhasilan OCR per Kategori (Heatmap)
    pivot_df = df.pivot_table(index='experiment', columns='ground_truth', values='is_correct', aggfunc='sum')
    
    plt.figure(figsize=(14, 8))
    sns.heatmap(pivot_df, annot=True, fmt='g', cmap='YlGnBu', linewidths=.5)
    plt.title('Heatmap Keberhasilan OCR (Jumlah Gambar yang Lolos)', fontsize=16)
    plt.xlabel('Jenis Onomatope', fontsize=12)
    plt.ylabel('Eksperimen', fontsize=12)
    plt.tight_layout()
    plot_path_heatmap = os.path.join(OUTPUT_DIR_PLOTS, 'ocr_success_heatmap.png')
    plt.savefig(plot_path_heatmap)
    plt.close()
    print(f"Heatmap keberhasilan disimpan di: {plot_path_heatmap}")

    # --- SECTION SUMMARY ---
    overall_accuracy = df['is_correct'].mean() * 100
    print("\n--- OCR Evaluation Complete ---")
    print(f"Hasil detail disimpan ke {OUTPUT_CSV}")
    print(f"Akurasi OCR Keseluruhan: {overall_accuracy:.2f}%")
    print("\nAkurasi per eksperimen:")
    print(accuracy_by_exp.to_string(index=False))


--- Generating Visualizations ---
Bar chart performa terbaik disimpan di: output_plots\best_experiment_ocr_performance.png


C:\Users\Pongo\AppData\Local\Temp\ipykernel_15292\4136363224.py:19: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=accuracy_by_exp, x='experiment', y='accuracy_%', palette='viridis')


Heatmap keberhasilan disimpan di: output_plots\ocr_success_heatmap.png

--- OCR Evaluation Complete ---
Hasil detail disimpan ke ocr_accuracy_results.csv
Akurasi OCR Keseluruhan: 4.00%

Akurasi per eksperimen:
       experiment  accuracy_%
  exp5_sdxl_a6000    8.333333
   exp1_sd15_2050    5.000000
   exp2_sd15_3050    3.333333
exp3_sd15_3050_v2    3.333333
exp4_sd15_3050_v3    0.000000
